In [1]:
%pip install "torch==2.4.0" "torchvision==0.19.0" "torchaudio==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 136.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 171.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 208.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 189.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 213.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 173.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 207.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 199.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 189.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvtx-cu12
    Found existing 

In [4]:
!pip install rich


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


# 1. 데이터 전처리

In [5]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [6]:
dataset = load_dataset("HJUNN/crypto_function_calling_datasets", split="train")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/319k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/384 [00:00<?, ? examples/s]

In [7]:
# 테스트 비율 설정
test_ratio = 0.2

# 전체 길이와 테스트 데이터 크기 계산
total_len = len(dataset)
test_size = int(total_len * test_ratio)

# 앞에서부터 테스트 데이터, 나머지는 학습 데이터
test_indices = list(range(test_size))
train_indices = list(range(test_size, total_len))

In [8]:
# OpenAI 포맷으로 변환 함수
def format_conversations(sample):
    return {
        "messages": [
            {"role": "system", "content": sample["system_prompt"]},
            *sample["messages"]
        ]
    }

# 분할 및 변환
train_dataset = [format_conversations(dataset[i]) for i in train_indices]
test_dataset = [format_conversations(dataset[i]) for i in test_indices]

# 리스트를 다시 HuggingFace Dataset 객체로 변환
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

# 결과 확인
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 308개, Test 76개


In [9]:
train_dataset[145]["messages"]

[{'content': '당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.\n\n로그인한 사용자의 현재 ID: U004\n오늘 날짜: 2024-02-26\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_latest_strategy", "description": "Fetch the latest N trading strategy records", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "limit": {"type": "integer", "default": 5}}, "required": ["userid"]}}}\n{"type": "function", "function": {"name": "get_top_movers", "description": "Get top price gainers", "parameters": {"type": "object", "properties": {"top_n": {"type": "integer", "default": 5}}}}}\n{"type": "function", "function": {"name": "get_strategy_by_date", "description": "Fetch trading history filtered by date or date range.", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "start_date": {"type": "string", "format": "date-t

# 2. 모델 로드 및 템플릿 적용

In [10]:
model_id = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map = "auto",
    torch_dtype = torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [11]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[145]["messages"], tokenize = False, add_generation_prompt = False
)
print(text)

<|im_start|>system
당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U004
오늘 날짜: 2024-02-26

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_latest_strategy", "description": "Fetch the latest N trading strategy records", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "limit": {"type": "integer", "default": 5}}, "required": ["userid"]}}}
{"type": "function", "function": {"name": "get_top_movers", "description": "Get top price gainers", "parameters": {"type": "object", "properties": {"top_n": {"type": "integer", "default": 5}}}}}
{"type": "function", "function": {"name": "get_strategy_by_date", "description": "Fetch trading history filtered by date or date range.", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "start_date": {"type": "string", "format": "date-time", "d

# 3. LoRA와 STFConfig 설정

In [12]:
peft_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout = 0.1,
    r = 8,
    bias = "none",
    target_modules = ["q_proj", "v_proj"],
    task_type = "CAUSAL_LM"
)

In [13]:
args = SFTConfig(
    output_dir="qwen2.5-7b-function-calling",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to= None,
)

# 4. 학습 중 전처리 함수: collate_fn

In [14]:
import torch

def collate_fn(batch):
    new_batch = {
        "input_ids" : [],
        "attention_mask" : [],
        "labels" : []
    }

    start_token = "<|im_start|>"
    end_token = "<im_end>"

    assistant_prefix = f"{start_token}assistant\n"
    assistant_tokens = tokenizer.encode(assistant_prefix, add_special_tokens = False)
    end_tokens = tokenizer.encode(end_token, add_special_tokens = False)

    for example in batch:
        messages = example["messages"]

        prompt = ""
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"{start_token}{role}\n{content}{end_token}"

        tokenized = tokenizer(
            prompt,
            truncation = True,
            max_length = max_seq_length,
            padding = False,
            return_tensors = None
        )
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 세그먼트 레이블에 복사(종료 토큰 포함)
        i = 0
        n = len(input_ids)
        while i <= n - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start_idx = i + len(assistant_tokens)
                end_idx = start_idx
                # <|im_end|>
                while end_idx <= n - len(end_tokens):
                    if input_ids[end_idx:end_idx + len(end_tokens)] == end_tokens:
                        end_idx += len(end_tokens)
                        break
                    end_idx += 1
                # start_idx 부터 end_idx 직전까지
                for j in range(start_idx, end_idx):
                    labels[j] = input_ids[j]
                i = end_idx
            else:
                i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 및 Tensor 변환
    max_len = max(len(ids) for ids in new_batch["input_ids"])
    for idx in range(len(new_batch["input_ids"])):
        pad_len = max_len - len(new_batch["input_ids"][idx])
        new_batch["input_ids"][idx].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][idx].extend([0] * pad_len)
        new_batch["labels"][idx].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

In [15]:
max_seq_length = 8192

example = train_dataset[0]
batch = collate_fn([example])

print("\n 처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


 처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 1607])
어텐션 마스크 형태: torch.Size([1, 1607])
레이블 형태: torch.Size([1, 1607])


In [16]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 35509, 55902, 64577, 85057, 3315, 109, 245, 144415, 58034, 125786, 55054, 78952, 13, 128677, 125512, 32831, 20401, 144484, 58034, 125786, 16186, 139713, 382, 81650, 31328, 23573, 40720, 131958, 132270, 3034, 25, 547, 15, 15, 19, 198, 57268, 127478, 37195, 254, 137771, 25, 220, 17, 15, 17, 18, 12, 16, 15, 12, 15, 20, 271, 2, 13852, 271, 2610, 1231, 1618, 825, 476, 803, 5746, 311, 7789, 448, 279, 1196, 3239, 382, 2610, 525, 3897, 448, 729, 32628, 2878, 366, 15918, 1472, 15918, 29, 11874, 9492, 510, 27, 15918, 397, 4913, 1313, 788, 330, 1688, 497, 330, 1688, 788, 5212, 606, 788, 330, 455, 3317, 13789, 497, 330, 4684, 788, 330, 20714, 2480, 1196, 5526, 504, 19565, 5952, 497, 330, 13786, 788, 5212, 1313, 788, 330, 1700, 497, 330, 13193, 788, 5212, 20085, 788, 5212, 1313, 788, 330, 917, 9207, 2137, 330, 6279, 788, 4383, 20085, 1341, 3417, 532, 4913, 1313, 788, 330, 1688, 497, 330, 1688, 788, 5212, 606, 788, 330, 18948, 55752, 497, 33

In [17]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens = False,
    clean_up_tokenization_spaces = False
)

print("\n input_ids 디코딩 결과:")
print(decoded_text)


 input_ids 디코딩 결과:
<|im_start|>system
당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U004
오늘 날짜: 2023-10-05

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_user_profile", "description": "Fetch full user profile from portfolio DB", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}}, "required": ["userid"]}}}
{"type": "function", "function": {"name": "compare_symbols", "description": "Compare multiple coins", "parameters": {"type": "object", "properties": {"symbols": {"type": "array", "items": {"type": "string"}}}, "required": ["symbols"]}}}
{"type": "function", "function": {"name": "get_latest_strategy", "description": "Fetch the latest N trading strategy records", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "limit": {"type": "integer", "default": 5}}, "required":

In [18]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens = False,
    clean_up_tokenization_spaces = False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
<tool_call>
{"name": "get_trending_coins", "arguments": {}}
</tool_call><im_end>현재 Dogecoin과 Shiba Inu가 많이 검색되고 있습니다.<im_end><|im_start|>user
그럼 비트코인이랑 이더리움 가격 비교해줘<im_end><tool_call>
{"name": "compare_symbols", "arguments": {"symbols": ["bitcoin", "ethereum"]}}
</tool_call><im_end>비트코인은 현재 약 $54,000로 24시간 동안 2.5% 상승했으며, 이더리움은 약 $3,500로 1.8% 상승했습니다.<im_end><|im_start|>user
지난주 내 거래 내역 좀 보여줘<im_end><tool_call>
{"name": "get_strategy_by_date", "arguments": {"userid": "U004", "start_date": "2023-09-28T00:00:00Z", "end_date": "2023-10-05T00:00:00Z"}}
</tool_call><im_end>지난주에는 비트코인을 매수하고, 이더리움을 매도하신 기록이 있습니다.<im_end><|im_start|>user
짝사랑 했던 그가 떠나갔어.<im_end><|im_start|>assistant
죄송하지만, 해당 질문에는 답변할 수 없습니다. 가상자산 시장 정보와 데이터 관련 질문만 도와드릴 수 있습니다.

추가로 궁금하신 가상자산 정보가 있으신가요?<


# 5. 학습

In [19]:
trainer = SFTTrainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    data_collator = collate_fn,
    peft_config = peft_config
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [20]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.136800
20,0.780600
30,0.639300
40,0.551400
50,0.478600
60,0.396000
70,0.379300
80,0.365800
90,0.331800
100,0.341700


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-pac

# 6. 테스테 데이터 준비하기

In [21]:
import re
from typing import List, Dict

def to_chatml(data):
    """
    data: messages 리스트이거나 {"messages": [...]} 형태의 dict
    반환값: ChatML 포맷의 문자열
    """
    # data가 dict이고 'messages' 키가 있으면 messages 리스트를 꺼내고,
    # 아니면 data 자체를 messages 리스트로 간주
    messages = data.get("messages") if isinstance(data, dict) and "messages" in data else data

    parts = []
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        parts.append(f"<|im_start|>{role}\n{content}<|im_end|>")
    return "\n".join(parts)

In [22]:
print(test_dataset[0])

{'messages': [{'content': '당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.\n\n로그인한 사용자의 현재 ID: U004\n오늘 날짜: 2023-07-25\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_user_profile", "description": "Fetch full user profile from portfolio DB", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}}, "required": ["userid"]}}}\n{"type": "function", "function": {"name": "get_latest_strategy", "description": "Fetch the latest N trading strategy records", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "limit": {"type": "integer", "default": 5}}, "required": ["userid"]}}}\n{"type": "function", "function": {"name": "search_crypto_term", "description": "Search crypto/economic terminology definitions from RAG database", "parameters": {"type": "object", "properties": {"query": {"type": "string"

In [23]:
chatml_test_dataset = to_chatml(test_dataset[0])
print(chatml_test_dataset)

<|im_start|>system
당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U004
오늘 날짜: 2023-07-25

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_user_profile", "description": "Fetch full user profile from portfolio DB", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}}, "required": ["userid"]}}}
{"type": "function", "function": {"name": "get_latest_strategy", "description": "Fetch the latest N trading strategy records", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "limit": {"type": "integer", "default": 5}}, "required": ["userid"]}}}
{"type": "function", "function": {"name": "search_crypto_term", "description": "Search crypto/economic terminology definitions from RAG database", "parameters": {"type": "object", "properties": {"query": {"type": "string"}, "top_k": {"type": 

In [24]:
def extract_examples(chatml: str) -> List[Dict[str, str]]:
    """
    ChatML 문자열에서 각 assistant 응답을 분리하여
    'input'과 'label' 쌍을 생성합니다.
    'input'은 해당 assistant 응답 직전까지의 모든 대화 + '<|im_start|>assistant',
    'label'은 해당 assistant의 응답 내용입니다.
    """
    examples: List[Dict[str, str]] = []
    pattern = re.compile(r'<\|im_start\|>assistant(.*?)(?=<\|im_end\|>)', re.DOTALL)

    for match in pattern.finditer(chatml):
        start_idx = match.start()
        input_text = chatml[:start_idx].strip() + '\n<|im_start|>assistant'
        label_text = match.group(1).strip()
        examples.append({
            "input": input_text,
            "label": label_text
        })

    return examples

In [25]:
prompt_lst = []
label_lst = []

for item in test_dataset:
    chatml = to_chatml(item)  # ChatML 문자열로 변환
    examples = extract_examples(chatml)  # assistant 응답 단위로 분리

    for ex in examples:
        prompt_lst.append(ex['input'])
        label_lst.append(ex['label'])

In [26]:
print('기존 테스트 데이터의 개수:', len(test_dataset))
print('턴 별 분리 후 테스트 데이터의 개수:', len(prompt_lst))

기존 테스트 데이터의 개수: 76
턴 별 분리 후 테스트 데이터의 개수: 585


In [27]:
print(prompt_lst[10])

<|im_start|>system
당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U002
오늘 날짜: 2025-07-26

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_trending_coins", "description": "Get trending coins", "parameters": {"type": "object", "properties": {}}}}
{"type": "function", "function": {"name": "get_crypto_news", "description": "Search latest crypto news from RAG DB", "parameters": {"type": "object", "properties": {"query": {"type": "string"}, "top_k": {"type": "integer", "default": 5}}, "required": ["query"]}}}
{"type": "function", "function": {"name": "get_latest_strategy", "description": "Fetch the latest N trading strategy records", "parameters": {"type": "object", "properties": {"userid": {"type": "string"}, "limit": {"type": "integer", "default": 5}}, "required": ["userid"]}}}
{"type": "function", "function": {"name": "get_mark

In [28]:
print(label_lst[10])

<tool_call>
{"name": "search_crypto_term", "arguments": {"query": "샤프 지수", "top_k": 1}}
</tool_call>


# 7. 파인튜닝 모델 테스트

In [37]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "qwen2.5-7b-function-calling/checkpoint-231"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map = "auto",
    torch_dtype = torch.float16,
    low_cpu_mem_usage = True
)
pipe = pipeline("text-generation", model = fine_tuned_model, tokenizer = tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [41]:
eos_token = tokenizer("<|im_end|>", add_special_tokens = False)["input_ids"][0]

In [42]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens = 1024, eos_token_id = eos_token, do_sample = False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [43]:
# 파인 튜닝 모델 호출
for prompt, label in zip(prompt_lst[70:75], label_lst[70:75]):
    print(f"    user:\n{prompt}")
    print('--' * 10)
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print('--' * 10)
    print(f"    label:\n{label}")
    print("=="*50)


    user:
<|im_start|>system
당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U002
오늘 날짜: 2024-04-07

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_crypto_news", "description": "Search latest crypto news from RAG DB", "parameters": {"type": "object", "properties": {"query": {"type": "string"}, "top_k": {"type": "integer", "default": 5}}, "required": ["query"]}}}
{"type": "function", "function": {"name": "get_24h_stats", "description": "Get 24-hour trading statistics", "parameters": {"type": "object", "properties": {"symbol": {"type": "string"}}, "required": ["symbol"]}}}
{"type": "function", "function": {"name": "get_price", "description": "Get real-time crypto price and market cap", "parameters": {"type": "object", "properties": {"symbol": {"type": "string"}}, "required": ["symbol"]}}}
{"type": "function", "function": {"nam

# 8. 기본 모델 테스트

In [44]:
base_model_id = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [46]:
# 기본 모델 호출
for prompt, label in zip(prompt_lst[70:75], label_lst[70:75]):
    print(f"    user:\n{prompt}")
    print('--' * 10)
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print('--' * 10)
    print(f"    label:\n{label}")
    print("=="*50)

    user:
<|im_start|>system
당신은 가상 자산 챗봇 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U002
오늘 날짜: 2024-04-07

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_crypto_news", "description": "Search latest crypto news from RAG DB", "parameters": {"type": "object", "properties": {"query": {"type": "string"}, "top_k": {"type": "integer", "default": 5}}, "required": ["query"]}}}
{"type": "function", "function": {"name": "get_24h_stats", "description": "Get 24-hour trading statistics", "parameters": {"type": "object", "properties": {"symbol": {"type": "string"}}, "required": ["symbol"]}}}
{"type": "function", "function": {"name": "get_price", "description": "Get real-time crypto price and market cap", "parameters": {"type": "object", "properties": {"symbol": {"type": "string"}}, "required": ["symbol"]}}}
{"type": "function", "function": {"nam